# Assignment  ( Big Data )


##  Setting-Up Configuration For Spark Environment

In [1]:

# configuring pyspark environment



import os
import sys

os.environ["SPARK_HOME"] = "/usr/spark2.4.3"
os.environ["PYLIB"] = os.environ["SPARK_HOME"] + "/python/lib"
os.environ["PYSPARK_PYTHON"] = "/usr/local/anaconda/bin/python" 
os.environ["PYSPARK_DRIVER_PYTHON"] = "/usr/local/anaconda/bin/python"
sys.path.insert(0, os.environ["PYLIB"] +"/py4j-0.10.7-src.zip")
sys.path.insert(0, os.environ["PYLIB"] +"/pyspark.zip")
 


##   Importing Required Libraries

In [2]:

import pyspark
                                             
    
# importing spark context and spark conf to create rdd's

from pyspark import SparkContext, SparkConf   
    
# Defining sql context through spark session for creating dataframes
                  
from pyspark.sql import SparkSession, SQLContext   
       
# For executing sql funtions

from pyspark.sql.functions import *
from pyspark.sql.functions import col, lit, when
from pyspark.sql import functions as f
from pyspark.sql.types import *
from pyspark.sql import Row


In [3]:

# Configuring spark session and sql context to create dataframes


spark = SparkSession.builder.getOrCreate()
sqlContext = SQLContext(spark)



In [4]:


#  Strategy:


# There are two types of data files  1)  Average_house price data  2) Price_Percentage_change(yearly) data


# Mutliple csv files of Average_house price data are loaded into one dataframe and Data-pre-processing is performed.

# Mutliple csv files of Price_Percentage_change(yearly) data are loaded into one dataframe and Data-pre-processing is performed.
                                         
#  Exploratory data analysis is performed using the dataframes obtained after data Pre-processing 



#  Data Pre-processing On Average House  Price Data
 


## Importing Data

In [5]:

# Creating a DataFrame directly from multiple csv files containing Average Price Data 



data_average_price_1 = spark.read.option("header", "true").option("delimiter", ",")\
                       .option("inferSchema", "true").csv("hdfs:///user/imat5322_1308287/*_price.csv",)


In [6]:

 # Calling an action to Look into the data loaded

data_average_price_1.show(2)


+------------+--------------------+---------------+-------+------------+----------------+--------------------------------+-----------------------------+----------------------------------+-----------------------------+-----------------------------------+--------------+
|        Name|                 URI|Region GSS code| Period|Sales volume|Reporting period|Average price All property types|Average price Detached houses|Average price Semi-detached houses|Average price Terraced houses|Average price Flats and maisonettes|Pivotable date|
+------------+--------------------+---------------+-------+------------+----------------+--------------------------------+-----------------------------+----------------------------------+-----------------------------+-----------------------------------+--------------+
|Inner London|http://landregist...|      E13000001|2017-01|        2836|         monthly|                          579471|                      1350726|                            897741|      

In [7]:

 # Calling an action to Look into number of records


data_average_price_1.count()


121

In [8]:

# Printing Average price data schema


data_average_price_1.printSchema()


root
 |-- Name: string (nullable = true)
 |-- URI: string (nullable = true)
 |-- Region GSS code: string (nullable = true)
 |-- Period: string (nullable = true)
 |-- Sales volume: string (nullable = true)
 |-- Reporting period: string (nullable = true)
 |-- Average price All property types: string (nullable = true)
 |-- Average price Detached houses: string (nullable = true)
 |-- Average price Semi-detached houses: string (nullable = true)
 |-- Average price Terraced houses: string (nullable = true)
 |-- Average price Flats and maisonettes: string (nullable = true)
 |-- Pivotable date: string (nullable = true)



## Filtering Columns

In [9]:

# Dropping columns which are not required from the data  

data_average_price_1 = data_average_price_1.drop('URI','Region GSS code', 'Reporting period','Pivotable date' )


In [10]:

# Calling an action to Look into the dataset columns

data_average_price_1.columns


['Name',
 'Period',
 'Sales volume',
 'Average price All property types',
 'Average price Detached houses',
 'Average price Semi-detached houses',
 'Average price Terraced houses',
 'Average price Flats and maisonettes']

## Renaming Columns

In [11]:

# Renaming the columns  /  tidy up the columns

data_average_price_1   = data_average_price_1.withColumnRenamed('Name','City')

data_average_price_1   = data_average_price_1.withColumnRenamed('Sales volume','Sales_volume')

data_average_price_1   = data_average_price_1.withColumnRenamed('Average price All property types',\
                                                                'Average_price_All_property_types')

data_average_price_1   = data_average_price_1.withColumnRenamed('Average price Detached houses',\
                                                                'Average_price_Detached_houses')

data_average_price_1   = data_average_price_1.withColumnRenamed('Average price Semi-detached houses',\
                                                                'Average_price_Semi_detached_houses')

data_average_price_1   = data_average_price_1.withColumnRenamed('Average price Terraced houses',\
                                                                'Average_price_Terraced_houses')

data_average_price_1   = data_average_price_1.withColumnRenamed('Average price Flats and maisonettes',\
                                                                'Average_price_Flats_and_maisonettes')


In [12]:

# Recheck to the renamed columns

data_average_price_1.columns



['City',
 'Period',
 'Sales_volume',
 'Average_price_All_property_types',
 'Average_price_Detached_houses',
 'Average_price_Semi_detached_houses',
 'Average_price_Terraced_houses',
 'Average_price_Flats_and_maisonettes']

## Handling Missing Values

In [13]:

# Checking for missing values :
                              
    
i = 0                                                                          # Introducing a counter         

                                                                               # Getting total records in the dataframe    

total = data_average_price_1.count()
print("Total Records = " + str(total))
                                                                               # Printing the amount of columns
print("Total columns = " + str(len(data_average_price_1.columns)))        
print("------------------------------------------")

                                                                           # looping entire df to get  missing value number 
                                                                           # and missing value number of each column
while i < len(data_average_price_1.columns):
                                                                               # loop through all columns    
    print(str(i+1) + "." + str(data_average_price_1[i]))
    print("  Missing Values = ") 
    print("------------------------------------------")    
                                                                               # check for missing values     
        
        
    missing_value = data_average_price_1.select([count(when(data_average_price_1[i].isNull(),True))]).show()   
    
                                                                              # adding counter by 1   
    i = i+1         
    

Total Records = 121
Total columns = 8
------------------------------------------
1.Column<b'City'>
  Missing Values = 
------------------------------------------
+---------------------------------------------+
|count(CASE WHEN (City IS NULL) THEN true END)|
+---------------------------------------------+
|                                            1|
+---------------------------------------------+

2.Column<b'Period'>
  Missing Values = 
------------------------------------------
+-----------------------------------------------+
|count(CASE WHEN (Period IS NULL) THEN true END)|
+-----------------------------------------------+
|                                              1|
+-----------------------------------------------+

3.Column<b'Sales_volume'>
  Missing Values = 
------------------------------------------
+-----------------------------------------------------+
|count(CASE WHEN (Sales_volume IS NULL) THEN true END)|
+-----------------------------------------------------+
|     

In [14]:

# Removing records which have Null values


data_average_price_2 = data_average_price_1.na.drop(how = "any")


In [15]:

# Checking number of records after droping null values


print('Before dropping : '+ str(data_average_price_1.count()))
print(" ")    
print('After  dropping : '+ str(data_average_price_2.count()))


Before dropping : 121
 
After  dropping : 118


In [16]:

# Checking for missing values after droping the "nan" values



                                                                            # Introducing a counter
    
i = 0               

                                                                            # Getting total records in the dataframe    

total = data_average_price_2.count()
print("Total Records = " + str(total))

                                                                            # Printing the amount of columns

print("Total columns = " + str(len(data_average_price_2.columns)))        
print("------------------------------------------")

                                        # looping entire df and get the missing value number and missing rate of each column

while i < len(data_average_price_2.columns):    
                                                                            #loop through all columns    
    print(str(i+1) + "." + str(data_average_price_2[i]))
    print("  Missing Values = ") 
    print("------------------------------------------")    
                                                                            # check for missing values     

    missing_value = data_average_price_2.select([count(when(data_average_price_2[i].isNull(),True))]).show()   
    
                                                                            # adding counter by 1   

    i = i+1                       
    

Total Records = 118
Total columns = 8
------------------------------------------
1.Column<b'City'>
  Missing Values = 
------------------------------------------
+---------------------------------------------+
|count(CASE WHEN (City IS NULL) THEN true END)|
+---------------------------------------------+
|                                            0|
+---------------------------------------------+

2.Column<b'Period'>
  Missing Values = 
------------------------------------------
+-----------------------------------------------+
|count(CASE WHEN (Period IS NULL) THEN true END)|
+-----------------------------------------------+
|                                              0|
+-----------------------------------------------+

3.Column<b'Sales_volume'>
  Missing Values = 
------------------------------------------
+-----------------------------------------------------+
|count(CASE WHEN (Sales_volume IS NULL) THEN true END)|
+-----------------------------------------------------+
|     

## Handling the schema of the dataframe

In [17]:

# looking into the original schema of the data

data_average_price_2.printSchema()


root
 |-- City: string (nullable = true)
 |-- Period: string (nullable = true)
 |-- Sales_volume: string (nullable = true)
 |-- Average_price_All_property_types: string (nullable = true)
 |-- Average_price_Detached_houses: string (nullable = true)
 |-- Average_price_Semi_detached_houses: string (nullable = true)
 |-- Average_price_Terraced_houses: string (nullable = true)
 |-- Average_price_Flats_and_maisonettes: string (nullable = true)



In [18]:

# Casting the schema of the columns  


data_average_price_2 = data_average_price_2.withColumn('City',data_average_price_2['City'].cast(StringType()))

data_average_price_2 = data_average_price_2.withColumn('Period',data_average_price_2['Period'].cast(StringType()))

data_average_price_2 = data_average_price_2.withColumn('Sales_volume',data_average_price_2['Sales_volume']\
                                                       .cast(IntegerType()))

data_average_price_2 = data_average_price_2.withColumn('Average_price_All_property_types',data_average_price_2\
                                                       
                                                       ['Average_price_All_property_types'].cast(IntegerType()))

data_average_price_2 = data_average_price_2.withColumn('Average_price_Detached_houses',data_average_price_2\
                                                       
                                                       ['Average_price_Detached_houses'].cast(IntegerType()))

data_average_price_2 = data_average_price_2.withColumn('Average_price_Semi_detached_houses',data_average_price_2\
                                                       
                                                       ['Average_price_Semi_detached_houses'].cast(IntegerType()))

data_average_price_2 = data_average_price_2.withColumn('Average_price_Terraced_houses',data_average_price_2\
                                                       
                                                       ['Average_price_Terraced_houses'].cast(IntegerType()))

data_average_price_2 = data_average_price_2.withColumn('Average_price_Flats_and_maisonettes',data_average_price_2\
                                                       
                                                       ['Average_price_Flats_and_maisonettes'].cast(IntegerType()))



In [19]:

# checking for schema changed


data_average_price_2.printSchema()


root
 |-- City: string (nullable = true)
 |-- Period: string (nullable = true)
 |-- Sales_volume: integer (nullable = true)
 |-- Average_price_All_property_types: integer (nullable = true)
 |-- Average_price_Detached_houses: integer (nullable = true)
 |-- Average_price_Semi_detached_houses: integer (nullable = true)
 |-- Average_price_Terraced_houses: integer (nullable = true)
 |-- Average_price_Flats_and_maisonettes: integer (nullable = true)



In [20]:

# Changing the schema of 'Period' from string to date and saving into new column 'Time_Period'


data_average_price_3 = data_average_price_2.withColumn("Time_Period",f.to_date('Period','yyyy-MM'))


In [21]:

# printing schema

data_average_price_3.printSchema()

root
 |-- City: string (nullable = true)
 |-- Period: string (nullable = true)
 |-- Sales_volume: integer (nullable = true)
 |-- Average_price_All_property_types: integer (nullable = true)
 |-- Average_price_Detached_houses: integer (nullable = true)
 |-- Average_price_Semi_detached_houses: integer (nullable = true)
 |-- Average_price_Terraced_houses: integer (nullable = true)
 |-- Average_price_Flats_and_maisonettes: integer (nullable = true)
 |-- Time_Period: date (nullable = true)



In [22]:

# looking into the data to check 'Time_Period' field introduced

data_average_price_3.show(118)


+------------+-------+------------+--------------------------------+-----------------------------+----------------------------------+-----------------------------+-----------------------------------+-----------+
|        City| Period|Sales_volume|Average_price_All_property_types|Average_price_Detached_houses|Average_price_Semi_detached_houses|Average_price_Terraced_houses|Average_price_Flats_and_maisonettes|Time_Period|
+------------+-------+------------+--------------------------------+-----------------------------+----------------------------------+-----------------------------+-----------------------------------+-----------+
|Inner London|2017-01|        2836|                          579471|                      1350726|                            897741|                       678193|                             537324| 2017-01-01|
|Inner London|2017-02|        2787|                          582285|                      1347727|                            896440|                   

In [23]:


# We have performed pre processing for the Average_house_price dataset.

# data_average_price_3 is the data frame obtained after pre-processing

# Below is the schema of the dataset


data_average_price_3.printSchema()

root
 |-- City: string (nullable = true)
 |-- Period: string (nullable = true)
 |-- Sales_volume: integer (nullable = true)
 |-- Average_price_All_property_types: integer (nullable = true)
 |-- Average_price_Detached_houses: integer (nullable = true)
 |-- Average_price_Semi_detached_houses: integer (nullable = true)
 |-- Average_price_Terraced_houses: integer (nullable = true)
 |-- Average_price_Flats_and_maisonettes: integer (nullable = true)
 |-- Time_Period: date (nullable = true)





#  Data Pre-processing On Price_Percentage_Change(Yearly) Data
 


## Importing Data

In [24]:

# Creating a DataFrame directly from multiple csv files containing percentage change Data 


data_percentage_change_1 = spark.read.option("header", "true").option("delimiter", ",").option("inferSchema", "true")\
                           .csv("hdfs:///user/imat5322_1308287/*change.csv")


In [25]:

# Printing schema of the data


data_percentage_change_1.printSchema()


root
 |-- Name: string (nullable = true)
 |-- URI: string (nullable = true)
 |-- Region GSS code: string (nullable = true)
 |-- Period: string (nullable = true)
 |-- Sales volume: string (nullable = true)
 |-- Reporting period: string (nullable = true)
 |-- Percentage change (yearly) All property types: string (nullable = true)
 |-- Percentage change (yearly) Detached houses: string (nullable = true)
 |-- Percentage change (yearly) Semi-detached houses: string (nullable = true)
 |-- Percentage change (yearly) Terraced houses: string (nullable = true)
 |-- Percentage change (yearly) Flats and maisonettes: string (nullable = true)
 |-- Pivotable date: string (nullable = true)



In [26]:

# Calling an action to Look into number of records in the data set


data_percentage_change_1.count()


120

In [27]:

# Calling an action to Look into the data set


data_percentage_change_1.show(120)


+------------+--------------------+---------------+-------+------------+----------------+---------------------------------------------+------------------------------------------+-----------------------------------------------+------------------------------------------+------------------------------------------------+--------------+
|        Name|                 URI|Region GSS code| Period|Sales volume|Reporting period|Percentage change (yearly) All property types|Percentage change (yearly) Detached houses|Percentage change (yearly) Semi-detached houses|Percentage change (yearly) Terraced houses|Percentage change (yearly) Flats and maisonettes|Pivotable date|
+------------+--------------------+---------------+-------+------------+----------------+---------------------------------------------+------------------------------------------+-----------------------------------------------+------------------------------------------+------------------------------------------------+--------------

## Filtering Cloumns

In [28]:

# Dropping columns which are not required from the data  

data_percentage_change_2 = data_percentage_change_1.drop('URI', 'Reporting period','Region GSS code','Pivotable date')


In [29]:

# Calling Action to Check the columns in the data set.

data_percentage_change_2.columns


['Name',
 'Period',
 'Sales volume',
 'Percentage change (yearly) All property types',
 'Percentage change (yearly) Detached houses',
 'Percentage change (yearly) Semi-detached houses',
 'Percentage change (yearly) Terraced houses',
 'Percentage change (yearly) Flats and maisonettes']

## Renaming Columns

In [30]:

# Renaming the columns  /  tidy up the columns


data_percentage_change_2   = data_percentage_change_2.withColumnRenamed('Name','City')

data_percentage_change_2   = data_percentage_change_2.withColumnRenamed('Sales volume','Sales_volume')

data_percentage_change_2   = data_percentage_change_2.withColumnRenamed('Percentage change (yearly) All property types',\
                                                                        'Percentage_change_All_property_types')

data_percentage_change_2   = data_percentage_change_2.withColumnRenamed('Percentage change (yearly) Detached houses',\
                                                                        'Percentage_change_Detached_houses')

data_percentage_change_2   = data_percentage_change_2.withColumnRenamed('Percentage change (yearly) Semi-detached houses',\
                                                                        'Percentage_change_Semi-detached_houses')

data_percentage_change_2   = data_percentage_change_2.withColumnRenamed('Percentage change (yearly) Terraced houses',\
                                                                        'Percentage_change_Terraced_houses')

data_percentage_change_2   = data_percentage_change_2.withColumnRenamed('Percentage change (yearly) Flats and maisonettes',\
                                                                        'Percentage_change_Flats_and_maisonettes')


In [31]:

# Looking into columns to check columns in the data set

data_percentage_change_2.columns


['City',
 'Period',
 'Sales_volume',
 'Percentage_change_All_property_types',
 'Percentage_change_Detached_houses',
 'Percentage_change_Semi-detached_houses',
 'Percentage_change_Terraced_houses',
 'Percentage_change_Flats_and_maisonettes']

## Handling Missing Values

In [32]:

# Checking for missing values in the data 

                                                                                     # Introducing a counter    
i = 0               
                                                                                     # Getting total records in the dataframe
total = data_percentage_change_2.count()
print("Total Records = " + str(total))
                                                                                     # Printing the amount of columns
print("Total columns = " + str(len(data_percentage_change_2.columns)))        
print("------------------------------------------")

                                                                    # looping entire df and get the missing value number 
                                                                    # And missing rate of each column

while i < len(data_percentage_change_2.columns):    
                                                                    # loop through all columns    
    print(str(i+1) + "." + str(data_percentage_change_2[i]))
    print("  Missing Values = ") 
    print("------------------------------------------")    
                                                                    # check for missing values     

    missing_value = data_percentage_change_2.select([count(when(data_percentage_change_2[i].isNull(),True))]).show()   
    
                                                                    # Adding counter by 1   

    i = i+1                                                                   


Total Records = 120
Total columns = 8
------------------------------------------
1.Column<b'City'>
  Missing Values = 
------------------------------------------
+---------------------------------------------+
|count(CASE WHEN (City IS NULL) THEN true END)|
+---------------------------------------------+
|                                            0|
+---------------------------------------------+

2.Column<b'Period'>
  Missing Values = 
------------------------------------------
+-----------------------------------------------+
|count(CASE WHEN (Period IS NULL) THEN true END)|
+-----------------------------------------------+
|                                              0|
+-----------------------------------------------+

3.Column<b'Sales_volume'>
  Missing Values = 
------------------------------------------
+-----------------------------------------------------+
|count(CASE WHEN (Sales_volume IS NULL) THEN true END)|
+-----------------------------------------------------+
|     

In [33]:


# FROM THE ABOVE TABLE WE SEE THERE ARE NO RECORDS WHERE ALL VALUES ARE EMPTY. 

# BUT WE FIND Two NAN VALUES IN Sales_volume column. 


In [34]:

# Excluding all records with nan values in the data.

data_percentage_change_3 = data_percentage_change_2.na.drop(how = "any")


In [35]:

# Checking number of records after droping null values


print('Before dropping : '+ str(data_percentage_change_2.count()))
print(" ")    
print('After  dropping : '+ str(data_percentage_change_3.count()))


Before dropping : 120
 
After  dropping : 118


In [36]:
# Checking for missing values after droping the "nan" values



                                                                            # Introducing a counter
    
i = 0               

                                                                            # Getting total records in the dataframe    

total = data_percentage_change_3.count()
print("Total Records = " + str(total))

                                                                            # Printing the amount of columns

print("Total columns = " + str(len(data_percentage_change_3.columns)))        
print("------------------------------------------")

                                        # looping entire df and get the missing value number and missing rate of each column

while i < len(data_percentage_change_3.columns):    
                                                                            #loop through all columns    
    print(str(i+1) + "." + str(data_percentage_change_3[i]))
    print("  Missing Values = ") 
    print("------------------------------------------")    
                                                                            # check for missing values     

    missing_value = data_percentage_change_3.select([count(when(data_percentage_change_3[i].isNull(),True))]).show()   
    
                                                                            # adding counter by 1   

    i = i+1                       
    

Total Records = 118
Total columns = 8
------------------------------------------
1.Column<b'City'>
  Missing Values = 
------------------------------------------
+---------------------------------------------+
|count(CASE WHEN (City IS NULL) THEN true END)|
+---------------------------------------------+
|                                            0|
+---------------------------------------------+

2.Column<b'Period'>
  Missing Values = 
------------------------------------------
+-----------------------------------------------+
|count(CASE WHEN (Period IS NULL) THEN true END)|
+-----------------------------------------------+
|                                              0|
+-----------------------------------------------+

3.Column<b'Sales_volume'>
  Missing Values = 
------------------------------------------
+-----------------------------------------------------+
|count(CASE WHEN (Sales_volume IS NULL) THEN true END)|
+-----------------------------------------------------+
|     

## Handling the schema of the dataframe

In [37]:

# looking into the original schema of the data

data_percentage_change_3.printSchema()

root
 |-- City: string (nullable = true)
 |-- Period: string (nullable = true)
 |-- Sales_volume: string (nullable = true)
 |-- Percentage_change_All_property_types: string (nullable = true)
 |-- Percentage_change_Detached_houses: string (nullable = true)
 |-- Percentage_change_Semi-detached_houses: string (nullable = true)
 |-- Percentage_change_Terraced_houses: string (nullable = true)
 |-- Percentage_change_Flats_and_maisonettes: string (nullable = true)



In [38]:

# changing the schema of 'Period' from string to date and saving into new column 'Time_Period'


data_percentage_change_3 = data_percentage_change_3.withColumn("Time_Period",f.to_date('Period','yyyy-MM'))

In [39]:

# casting the schema for the columns 


data_percentage_change_3 = data_percentage_change_3.withColumn('City',data_percentage_change_3['City']\
                                                               .cast(StringType()))


data_percentage_change_3 = data_percentage_change_3.withColumn('Period',data_percentage_change_3['Period']\
                                                               .cast(StringType()))


data_percentage_change_3 = data_percentage_change_3.withColumn('Sales_volume',data_percentage_change_3['Sales_volume']\
                                                               .cast(IntegerType()))


data_percentage_change_3 = data_percentage_change_3.withColumn('Percentage_change_All_property_types'\
                                                               ,data_percentage_change_3['Percentage_change_All_property_types']\
                                                               .cast(FloatType()))


data_percentage_change_3 = data_percentage_change_3.withColumn('Percentage_change_Detached_houses'\
                                                               ,data_percentage_change_3['Percentage_change_Detached_houses']\
                                                               .cast(FloatType()))


data_percentage_change_3 = data_percentage_change_3.withColumn('Percentage_change_Semi-detached_houses',data_percentage_change_3\
                                                               ['Percentage_change_Semi-detached_houses'].cast(FloatType()))


data_percentage_change_3 = data_percentage_change_3.withColumn('Percentage_change_Terraced_houses',data_percentage_change_3\
                                                               ['Percentage_change_Terraced_houses'].cast(FloatType()))

data_percentage_change_3 = data_percentage_change_3.withColumn('Percentage_change_Flats_and_maisonettes',data_percentage_change_3\
                                                               ['Percentage_change_Flats_and_maisonettes'].cast(FloatType()))


In [40]:

data_percentage_change_3.printSchema()


root
 |-- City: string (nullable = true)
 |-- Period: string (nullable = true)
 |-- Sales_volume: integer (nullable = true)
 |-- Percentage_change_All_property_types: float (nullable = true)
 |-- Percentage_change_Detached_houses: float (nullable = true)
 |-- Percentage_change_Semi-detached_houses: float (nullable = true)
 |-- Percentage_change_Terraced_houses: float (nullable = true)
 |-- Percentage_change_Flats_and_maisonettes: float (nullable = true)
 |-- Time_Period: date (nullable = true)



In [41]:
# counting number of records available

data_percentage_change_3.count()


118

In [42]:
# We have performed pre processing for the Price_Percentage_Change(Yearly) Dataset.


data_percentage_change_3.show(118)


+------------+-------+------------+------------------------------------+---------------------------------+--------------------------------------+---------------------------------+---------------------------------------+-----------+
|        City| Period|Sales_volume|Percentage_change_All_property_types|Percentage_change_Detached_houses|Percentage_change_Semi-detached_houses|Percentage_change_Terraced_houses|Percentage_change_Flats_and_maisonettes|Time_Period|
+------------+-------+------------+------------------------------------+---------------------------------+--------------------------------------+---------------------------------+---------------------------------------+-----------+
|Inner London|2017-01|        2836|                                2.89|                             2.47|                                  1.73|                             1.77|                                   3.28| 2017-01-01|
|Inner London|2017-02|        2787|                                2.99|

In [43]:

# Data-Pre-processing is done and we have 2 dataframes:    

# 1) data_average_price_3     2) data_percentage_change_3

# which contain data for analysis


# Exploratory Data Analysis & Visualization

In [44]:

# Data for analysis :

                    #  1) data_average_price_3     2) data_percentage_change_3



In [45]:

# Registering average house price data and percentage change house price data into tables to run sql queries for analysis.
    

# Average house price data

data_average_price_3.registerTempTable("Average_price_table")

# Percentage change house price data

data_percentage_change_3.registerTempTable("percentage_change_table")


In [46]:

# Importing pixiedust library for visualization


import pixiedust

Pixiedust database opened successfully


## Time-Series Analysis On House Sales during 2017, 2018, 2019 (Pre-lockdown)

In [47]:

    # Creating dataframe with sales data using sql quiries

    # Data Table used  = Average_price_table



Sales = sqlContext.sql('select City,substring(Period,1,4) as Year, Time_Period as Period,\
                        Sales_volume from Average_price_table' )   
           
           
           
           
                      

### Exploring Trend In House Sales In 2017 (Pre-lockdown) 

In [48]:

# Visualizing trend in sales during 2017

display(Sales)


In [49]:
display(Sales)

### Exploring Trend In House Sales In 2018 (Pre-lockdown) 

In [50]:

# Visualizing trend in sales during 2018.

display(Sales)

### Exploring Trend In House Sales In 2019 (Pre-lockdown) 

In [51]:

# Visualizing trend in sales during 2019.

display(Sales)

## Time-Series Analysis On House Sales 2020 (Lockdown Period)

In [52]:

# Visualizing trend in sales during 2020. 

display(Sales)

In [53]:
# we have seen that the sales had a sharp decrease at the start of the lockdown during march-april but 
# have gone up after march why this was the case?

# what was the price pattern during these months?
        # did prices decrease ? after march ? what was the price percentage change during the first 2 months of the lockdown
    


In [54]:


# so as per above discussion plotting the percentage change in prices during the 2019 and 2020 and looking into those months 

# seeing what hapened in percentage that first 2 months sales gone way too low and increased gradually.

##  House Price Analysis During 2019 & 2020

In [55]:

# Creating dataframe with Percentage Change In House Price data using sql quiries

# Data Table used  = percentage_change_table


Percentage_Change = sqlContext.sql('select City as City, substring(Period,1,4) as Year, Time_Period as Period,\
Percentage_change_All_property_types as Percentage_change from percentage_change_table')


In [56]:

# Visualizing Trend in prices

display(Percentage_Change)

In [57]:
# Visualizing Trend in prices during start of lockdown in 2020.

display(Percentage_Change)

In [58]:

# we find that at the start of the lock down the price of the houses were too high therefore low sales and 
# gradually the prices went low which made the sales to go up


# now we want to see the individual house type prices


## Analysis on Individual house Type Price

In [59]:

# Creating dataframe with house prices according to the type using sql quiries

# Data Table used  = Average_price_table



Individual_House_Type = sqlContext.sql('select City as City, substring(Period,1,4) as Year, Time_Period as Period, Average_price_Detached_houses as Detached_houses, Average_price_Semi_detached_houses as Semi_detached, Average_price_Terraced_houses as Terraced_houses, Average_price_Flats_and_maisonettes as Flats_and_maisonettes from Average_price_table')   


In [60]:
display(Individual_House_Type)

City,Year,Period,Detached_houses,Semi_detached,Terraced_houses,Flats_and_maisonettes
Inner London,2017,2017-01-01 00:00:00,1350726,897741,678193,537324
Inner London,2017,2017-02-01 00:00:00,1347727,896440,679694,540466
Inner London,2017,2017-04-01 00:00:00,1358216,898260,680420,543278
Inner London,2017,2017-05-01 00:00:00,1359237,901742,679769,542917
Inner London,2017,2017-06-01 00:00:00,1358354,903064,680942,545709
Inner London,2017,2017-08-01 00:00:00,1381085,924059,695308,549200
Inner London,2017,2017-09-01 00:00:00,1388241,928042,697697,549338
Inner London,2017,2017-10-01 00:00:00,1378726,919532,691775,544761
Inner London,2017,2017-11-01 00:00:00,1352410,906109,681129,539417
Inner London,2017,2017-12-01 00:00:00,1340439,896512,674683,536553


## Time-Series Analysis On House Sales Overall (2017 - 2022)

In [61]:

# Visualizing overall trend in sales from 2017-2021 

# Pre-lockdown, Lockdown and Post-Lockdown


display(Sales)

In [62]:

# Visualizing overall Trend in prices from 2017-2021.

display(Percentage_Change)

In [ ]:


# From above graph it is evident that in both the cities 2021 was the year where the sales noticed an unusual\

# hike than any other years.